In [1]:
from sklearn.datasets import fetch_openml
mnist = fetch_openml('mnist_784', version=1, cache=True, as_frame=False)

In [2]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [3]:
X = mnist["data"]
y = mnist["target"].astype(np.int8)

In [4]:
X_train_full, X_test, y_train_full, y_test = X[:60000], X[60000:], y[:60000], y[60000:]


In [5]:
X_train, X_valid = train_test_split(X_train_full, test_size = 0.25, random_state = 42)
y_train, y_valid = train_test_split(y_train_full, test_size = 0.25, random_state = 42)

In [6]:
X_train_scaled = X_train / 255.0
X_test_scaled = X_test / 255.0
X_valid_scaled = X_valid / 255.0

In [7]:
X_train_scaled = X_train_scaled.reshape(len(X_train_scaled), 28, 28)
X_test_scaled = X_test_scaled.reshape(len(X_test_scaled), 28, 28)
X_valid_scaled = X_valid_scaled.reshape(len(X_valid), 28, 28)

DNN

In [32]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape = [28,28]),  
    keras.layers.Dropout(rate = 0.2), 
    keras.layers.Dense(300, activation = "elu", kernel_initializer = "he_normal"), #hidden1
    keras.layers.Dropout(rate = 0.2),
    keras.layers.Dense(100, activation = "elu", kernel_initializer = "he_normal"), #hidden2
    keras.layers.Dropout(rate=0.2),
    keras.layers.Dense(10, activation = "softmax",kernel_initializer = "glorot_normal"), #output
])

In [33]:
optimizer = keras.optimizers.Adam(learning_rate =0.001, beta_1=0.9, beta_2=0.999, decay=1e-4)

In [34]:
model.compile(loss = "sparse_categorical_crossentropy",
optimizer = optimizer , metrics = ["accuracy"])

In [35]:
checkpoint_cb = keras.callbacks.ModelCheckpoint("my_mnist_model_new.h5", 
                                                save_best_only=True)

Training

In [36]:
history = model.fit(X_train_scaled, y_train, epochs = 50,
validation_data = (X_valid_scaled, y_valid), batch_size = 32,
callbacks = [checkpoint_cb]
)

Epoch 1/50
1407/1407 [==============================] - 4s 2ms/step - loss: 0.3747 - accuracy: 0.8839 - val_loss: 0.1828 - val_accuracy: 0.9441
Epoch 2/50
1407/1407 [==============================] - 3s 2ms/step - loss: 0.2157 - accuracy: 0.9333 - val_loss: 0.1317 - val_accuracy: 0.9620
Epoch 3/50
1407/1407 [==============================] - 3s 2ms/step - loss: 0.1662 - accuracy: 0.9480 - val_loss: 0.1150 - val_accuracy: 0.9665
Epoch 4/50
1407/1407 [==============================] - 3s 2ms/step - loss: 0.1425 - accuracy: 0.9545 - val_loss: 0.0966 - val_accuracy: 0.9709
Epoch 5/50
1407/1407 [==============================] - 3s 2ms/step - loss: 0.1192 - accuracy: 0.9624 - val_loss: 0.0877 - val_accuracy: 0.9735
Epoch 6/50
1407/1407 [==============================] - 3s 2ms/step - loss: 0.1087 - accuracy: 0.9645 - val_loss: 0.0807 - val_accuracy: 0.9760
Epoch 7/50
1407/1407 [==============================] - 3s 2ms/step - loss: 0.0998 - accuracy: 0.9678 - val_loss: 0.0773 - val_accuracy:

In [37]:
model = keras.models.load_model("my_mnist_model_new.h5")
model.evaluate(X_test_scaled, y_test)

313/313 [==============================] - 0s 1ms/step - loss: 0.0517 - accuracy: 0.9850


[0.05168695002794266, 0.9850000143051147]

In [38]:
y_pred = np.argmax(model.predict(X_test_scaled), axis = -1) #predictions

313/313 [==============================] - 0s 945us/step


We finish the task with an accuracy of 98.50%